# PC5 - Sélection de modèle et régularisation - 27 juin 2025

Dans ce notebook, nous allons utiliser des données simulées pour mieux comprendre les __régularisations L1 et L2__. Ce sera aussi l'occasion de mettre en place une __recherche sur grille__ pour sélectionner le coefficient de régularisation par validation croisée. 

Dans la dernière partie, vous pourrez mettre en œuvre ces algorithmes sur un jeu de données réelles.

Ce notebook sera aussi l'occasion d'aborder un premier algorithme d'apprentissage supervisé _non-linéaire_, la __régression polynomiale__.

Ce notebook a été initialement proposé par [Arthur Imbert](https://github.com/Henley13).

### Import de numpy et matplotlib

In [ ]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt

In [ ]:
plt.rc('font', **{'size': 12}) # règle la taille de police globalement pour les plots (en pt)

In [ ]:
# On fixe ici la graine pour le générateur de nombres aléatoires, pour faciliter la reproducibilité
np.random.seed(19)

## 1. Régularisation L2 (ridge)

### 1.1 Simulation de données

Commençons par simuler un jeu de données de 30 échantillons avec une seule variable prédictive (p=1) et dans lequel l'étiquette est une fonction non-linéaire (sinusoïdale) de cette variable.

In [ ]:
nb_samples = 30

# vrai modèle (Y = true_f(X))
def true_f(x):
    return np.cos(1.5 * np.pi * x) * 5

# tirer nb_samples valeurs de x entre 0 et 1
X = np.random.rand(nb_samples, 1)
y = true_f(X)

# ajouter du bruit
y += np.random.randn(nb_samples, 1) * 0.3
print(X.shape, y.shape)

Nous pouvons maintenant visualiser ce que nous venons de simuler

In [ ]:
plt.figure(figsize=(6, 4))

# Pour afficher le vrai modèle :
# créer 100 points de vraies paires (x, y) 
# créer un array de dimension (1, 100) contenant 100 valeurs régulièrement espacées entre 0 et 1  
X_grid = np.linspace(0, 1, 100).reshape(-1, 1)
# calculer leurs étiquettes
y_true = true_f(X_grid)
plt.plot(X_grid, y_true, label="vérité", color="black", linewidth=1, linestyle='dashed')

# Afficher les données simulées
plt.scatter(X, y, label="observations simulées", color="tab:blue", marker="+", s=50)

# Mise en forme
plt.xlabel("$x$")
plt.ylabel("$y$")
plt.xlim((0, 1))
plt.ylim((-7, 7))

plt.title("Vrai modèle et données simulées")
plt.legend(loc="best")
plt.tight_layout()
plt.show()

Nous pouvons maintenant séparer nos données en un jeu d'entraînement et un jeu de test.

__Question :__ En s'inspirant de la PC4, séparer (`X`, `y`) en un jeu d'entraînement (`X_train`, `y_train`) et un jeu de test (`X_test`, `y_test`). Le jeu de test contiendra 30% des données.

__Question :__ Reproduire le graphique précédent, mais distinguer jeu d'entraînement (+) et jeu de test (x) parmi les observations simulées.

### 1.2 Régression linéaire classique

#### Entraînement du modèle

__Question :__ En s'inspirant de la PC4, entraîner une régression linéaire sur `(X_train, y_train)`. Appeler le modèle `linreg`.

Remarquez que les variables ayant été générées centrées-réduites, il n'est pas nécessaire de leur appliquer cette transformation.

Nous pouvons écrire explicitement le modèle appris en accédant à ses coefficients :

In [ ]:
print(f"L'équation du modèle appris est : y = {linreg.coef_[0][0]:.2f} x + {linreg.intercept_[0]:.2f}")

#### Performance du modèle

__Question :__ Calculer le [coefficient de détermination](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.r2_score.html) de `linreg` sur le jeu d'entraînement et sur le jeu de test.

__Question :__ Pourquoi comparer ces deux performances ? Qu'en conclure ici ?

__Réponse :__ Si la performance est moins bonne sur le jeu de test, cela indique une situation de sur-apprentissage. À l'inverse, si la performance est meilleure sur le jeu de test, cela indique un sous-apprentissage.

Ici, le modèle sous-apprend.

__Remarque :__ La différence entre les performances va dépendre d'où sont situées les observations de chacun des deux jeux.

Par exemple, si tous les points du jeu d'entraînement sont situés entre $x=0,2$ et $x=0,6$, le modèle est bien approché par une droite et la performance sur le jeu d'entraînement sera bonne. Cette droite est une mauvaise approximation de ce qui se passe à l'extérieur de cet intervalle, si le jeu de test contient des points qui s'y trouvent, la performance sera fortement dégradée.

#### Visualisation

__Question :__ Ajouter au graphique précédent le modèle appris.

### 1.3 Régression polynomiale 

Nous avons jusqu'à présent travaillé avec une seule variable.

Pour essayer d'améliorer notre modèle, nous pouvons _créer de nouvelles variables_ à partir de celle-ci (par exemple $x^2+x^3$, $\log(x)$, $e^{x-17}$) et apprendre un modèle linéaire sur ces nouvelles variables. Plutôt que de procéder à tatons comme dans les exemples entre parenthèse, on peut se limiter aux _puissances_ de notre variable $x$.

Ainsi, nous allons remplacer l'unique variable $x$ par $d$ variables $x, x^2, x^3, \dots, x^d$. Apprendre une fonction linéaire de ces $d$ variables est équivalent à apprendre un polynôme de degré $d$ de $x$. C'est une première approche pour apprendre un modèle non linéaire !

Cette idée se généralise à un nombre arbitraire de variables initiales ; on crée alors tous les _monomes_ de degré au plus $d$ de ces variables : $(x_1, x_2, \dots, x_p)$ devient $(x_1, x_2, \dots, x_p, x_1^2, x_1 x_2, \dots, x_p^d)$. On parle alors de _régression polynomiale_. Nous en reparlerons dans le chapitre 9. 

Dans scikit-learn, la classe [`PolynomialFeatures`](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.PolynomialFeatures.html) du module `preprocessing` permet de créer ces nouvelles variables, ce que nous allons faire ici pour un degré $d$=15 :

#### Entraînement du modèle

In [ ]:
from sklearn import preprocessing

In [ ]:
# Instancier un objet permettant de créer des variables polynomiales de degré au plus 15
polynomial_features = preprocessing.PolynomialFeatures(degree=15, include_bias=False)

# Appliquer cet objet aux variables de X_train 
X_train_poly = polynomial_features.fit_transform(X_train)

# Appliquer la même transformation aux données de test
X_test_poly = polynomial_features.transform(X_test)

# Ainsi qu'à la grille de points servant à appliquer le modèle sur tout [0, 1]
X_grid_poly = polynomial_features.transform(X_grid)

print(X_train_poly.shape, X_test_poly.shape, X_grid_poly.shape)

__Question :__ Combien de variables avons-nous maintenant ?

__Réponse :__ 

__Question :__ Entraîner maintenant une régression polynomiale `polyreg` sur le jeu d'entraînement.

__Question :__ Quelle est maintenant l'équation du modèle appris ?

__Question :__ Que dire de ces coefficients ?

__Réponse :__ 

#### Performance du modèle

__Question :__  Calculer le coefficient de détermination de la régression polynomiale sur le jeu d'entraînement et sur le jeu de test.

In [ ]:
# Calculer le coefficient de régression de polyreg sur le jeu d'entraînement
polyreg_r2_train = metrics.r2_score(y_train, polyreg.predict(X_train_poly))
print(f"R2 de la régression polynomiale sur le jeu d'entraînement : {polyreg_r2_train:.2f}.")

# Calculer le coefficient de régression de polyreg sur le jeu de test
polyreg_r2_test = metrics.r2_score(y_test, polyreg.predict(X_test_poly))
print(f"R2 de la régression polynomiale sur le jeu de test : {polyreg_r2_test:.2f}.")

__Question :__ Que pouvez-vous conclure sur le choix de la régression polynomiale ?

__Réponse :__ Le modèle surapprend. Les performances du jeu d'entraînement sont bien supérieures à celles du jeu de test.

#### Visualisation

__Question :__ Remplacer sur le graphique précédent le modèle appris par régression linéaire par celui  appris par régression polynomiale.

__Question :__ Le graphique est-il cohérent avec les performances calculées ?

__Réponse :__ 

### 1.4 Régression polynomiale régularisée ridge

Comme la régression polynomiale surapprend, nous allons maintenant lui appliquer un terme de __régularisation ridge (L2)__ pour essayer de compenser cet effet :

#### Entraînement du modèle

In [ ]:
# Instancier une régression linéaire avec régularisation ridge avec un coefficient de régularisation valant 0.01
polyreg_ridge = linear_model.Ridge(alpha=0.01, random_state=13)

__Question :__ Utiliser `polyreg_ridge` pour entraîner une régression polynomiale avec régularisation L2 sur le jeu d'entraînement.

__Question :__ Quelle est l'équation de ce modèle ?

__Question :__ Comparer ces coefficients à ceux du modèle appris sans régularisation.

__Réponse :__ 

#### Performance du modèle

__Question :__ Calculer le coefficient de détermination de la régression polynomiale avec régularisation ridge sur le jeu d'entraînement et sur le jeu de test.

__Question :__ Pensez-vous que le modèle surapprend toujours ?

__Réponse :__ 

#### Visualisation

__Question :__ Afficher maintenant ce nouveau modèle.

__Question :__ Le graphique est-il cohérent avec les performances calculées ?

__Réponse :__ 

__Question :__ Comment aurait-on pu essayer d'éviter le surapprentissage avec une régression polynomiale mais sans régularisation ?

__Réponse :__ 

## 2. Régularisation L1 (Lasso)

### 2.1 Simulation de données

Commençons par simuler un jeu de données de 60 échantillons avec 100 variables, et dans lequel l'étiquette est une fonction linéaire de seulement 10 de ces variables, les autres étant du bruit.

In [ ]:
nb_samples = 60
nb_features = 100

# créer un jeu de données aux dimensions demandées à partir d'une loi normale centrée-réduite
X = np.random.randn(nb_samples, nb_features)

# créer un vecteur de coefficients nuls
beta = np.zeros(nb_features)

# créer des coefficients pour les 10 premières variables
#   (pour faciliter la visualisation,
#   décroissants en valeur absolue, avec alternance de signe)
beta[:10] = [((-1) ** idx * np.exp(-idx/10)) for idx in range(10)]

# créer les étiquettes
y = np.dot(X, beta) + np.random.randn(nb_samples) * 0.1

Visualisons les coefficients du modèle ayant permis de simuler les données :

In [ ]:
plt.figure(figsize=(8, 5))

plt.stem(np.arange(nb_features), beta, markerfmt='o', 
         linefmt='tab:blue',
         label='vrais coefficients')

# Mise en forme
plt.xlabel("Variables")
plt.ylabel("Poids")
plt.title("Modèle parcimonieux")
plt.legend(loc="best")
plt.tight_layout()

__Question :__ Séparer (`X`, `y`) en un jeu d'entraînement (`X_train`, `y_train`) et un jeu de test (`X_test`, `y_test`). Le jeu de test contiendra 30% des données.

#### Corrélation entre les variables

La difficulté pour l'apprentissage est double :
- seule une faible proportion des variables influencent l'étiquette
- le nombre d'observations est faible par rapport à ce nombre de variables.

Un des problèmes qui apparait quand on a plus de variables que d'observations est que les variables peuvent apparaître corrélées même quand elles ne le sont pas. Affichons ici la matrice de corrélation entre les variables :

In [ ]:
import pandas as pd

In [ ]:
# Calcul de la matrice de corrélation deux à deux
df = pd.DataFrame(X_train)
X_train_corr = df.corr()

In [ ]:
import seaborn as sns

In [ ]:
plt.figure(figsize=(8, 6))
sns.heatmap(X_train_corr, annot=False, 
            vmin=-1, vmax=1, center=0, # bornes de la barre de couleurs
            cmap='PuOr', linewidths=0.5)
plt.title("Corrélation entre les variables de X_train")

__Question :__ Commenter cette matrice.

__Réponse :__ 

### 2.2 Régression linéaire

__Question :__ Entraîner sur (`X_train, y_train`) une régression linéaire classique. 

__Question :__ Évaluer la RMSE de cette régression linéaire sur le jeu d'entraînement et sur le jeu de test. La régression linéaire a-t-elle une performance satisfaisante ? Y-a-t'il un risque de sur- ou de sous-apprentissage ?

__Réponse :__ Il y a manifestement surapprentissage, la performance sur le jeu d'entraînement étant parfaite mais pas sur le jeu de test.

__Question :__ La performance sur le jeu d'entraînement est-elle surprenante ?

__Réponse :__ 

__Question :__ Ajouter à la visualisation des poids du modèle les coefficients appris par la régression linéaire.

__Question :__ Comparer les coefficients appris aux vrais coefficients.

__Réponse :__ 

### 2.3 Lasso

__Question :__ Entrainez un Lasso avec comme paramètre de régularisation `alpha=0.01` sur les données d'entraînement en utilisant la classe [`Lasso`](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Lasso.html) du module `linear_model`.

__Question :__ Évaluer la RMSE de ce lasso sur le jeu d'entraînement et sur le jeu de test. Y-a-t'il a un risque de sur- ou de sous-apprentissage ?

__Réponse :__ Il semble toujours y avoir du surapprentissage, mais moins que précédemment.

__Réponse :__ 

__Question :__ Ajouter à la visualisation des poids du modèle les coefficients appris par le lasso.

__Question :__ Comparer les coefficients appris aux vrais coefficients.

__Réponse :__ 

## 3. Sélection de modèle

Nous allons jusqu'à présent fixé la valeur du coefficient de régularisation (`alpha` dans `scikit-learn`, $\lambda$ dans le poly). Nous allons maintenant voir comment utiliser une recherche sur grille dans le cadre d'une validation croisée pour _sélectionner_ la valeur de ce coefficient, qui est un _hyperparmètre_ du lasso.

Nous travaillons toujours avec les données de la section 2.

### 3.1 Validation croisée

La classe `KFold` du module `model_selection` de scikit-learn permet de créer des _folds_ de validation croisée, c'est-à-dire de diviser un jeu de données en K blocs et de constituer K paires de jeux d'entraînement et de validation, où le jeu de validation est l'un des blocs et le jeu d'entraînement est l'union des (K-1) autres blocs.

In [ ]:
# instantier un objet KFold 
kf = model_selection.KFold(n_splits=5, shuffle=True, random_state=42)

La méthode `kf.split()` permet maintenant de partager un jeu de données en 5 folds. Attention, elle retourne un générateur, sur lequel on ne peut itérer qu'une fois. Fixer la valeur de `random_state` permet d'avoir la même partition à chaque appel.

In [ ]:
for i, (train_indices, val_indices) in enumerate(kf.split(X_train)):
    print(f"fold: {i} : {len(train_indices)} observations pour l'entraînement et {len(val_indices)} pour la validation.")

__Question :__ Compléter le code suivant pour déterminer la RMSE en validation croisée (c'est-à-dire la performance moyenne sur les 5 jeux de validation) d'un lasso avec coefficient de régularisation `alpha=0.01`.

In [ ]:
# Définir le modèle à évaluer
lasso = ...

rmse_list = []
# Boucler sur les folds :
for i, (train_indices, val_indices) in enumerate(kf.split(X_train)):
    print(f"fold: {i} : {len(train_indices)} observations pour l'entraînement et {len(val_indices)} pour la validation.")

    # créer le jeu d'entraînement et le jeu de validation pour ce fold
    X_train_fold = ...
    y_train_fold = ...
    X_test_fold = ...
    y_test_fold = ...

    # entraîner le modèle sur le jeu d'entraînement de ce fold
    lasso.fit(...)

    # prédire sur le jeu de validation du fold
    rmse_fold = ...
    rmse_list.append(rmse_fold)
    print(f"\tRMSE (test) : {rmse:.2f}")

# Moyenner les performances
rmse_average = np.mean(rmse_list)                    
print(f"La RMSE moyenne du Lasso (alpha=0.1) est de {rmse_average:.2f}")

Scikit-learn peut faire cette opération directement avec la fonction `cross_validate` du module `model_selection` :

In [ ]:
lasso_cv_scores = model_selection.cross_val_score(lasso, X_train, y_train, 
                                                  cv=kf, # utiliser les folds déjà définis 
                                                  scoring='neg_root_mean_squared_error')

Vérifions que l'on obtient bien les mêmes résultats :

In [ ]:
print(lasso_cv_scores)

In [ ]:
print(f"La RMSE moyenne du Lasso (alpha=0.01) est de {-np.mean(lasso_cv_scores):.2f}")

__Question :__ Pourquoi est-ce `neg_root_mean_squared_error`, qui retourne _l'opposé_ de la RMSE, qui est implémentée comme fonction de score et non pas directement la RMSE ?

__Réponse :__ Ces scores sont utilisés pour sélectionner le meilleur modèle, qui sera celui qui a le score de performance le plus élevé. Dans le cas de la RMSE, il s'agit donc de celui qui a la plus petite RMSE.

### 3.2 Recherche sur grille

La _recherche sur grille_ (_gridsearch_) consiste à comparer différentes valeurs d'une grille d'hyperparamètres en comparant la performance des modèles appris avec chacune de ces valeurs, généralement en utilisant une validation croisée.

Dans scikit-learn, cette procédure est implémentée dans la classe `GridSearchCV` de `model_selection` :

In [ ]:
# Définir la grille de valeurs de l'hyperparamètre alpha 
alphas = np.logspace(-5, 1, 40)

# Définir le modèle à évaluer
lasso = linear_model.Lasso(random_state=13, 
                           max_iter=10000 # pour assurer la convergence (warning sinon)
                          )

# Instantier la recherche sur grille
grid = model_selection.GridSearchCV(lasso, {'alpha': alphas}, 
                                    cv=kf, # on utilise les folds déjà définis
                                    scoring='neg_root_mean_squared_error'
                                   )

# Utiliser la recherche sur grille
grid.fit(X_train, y_train)

#### Valeur de la RMSE en fonction de alpha

Les détails des calculs effectués par `fit` sont accessibles dans le dictionnaire retourné par `grid.cv_results_` :

In [ ]:
grid.cv_results_.keys()

On peut ainsi récupérer les scores obtenus pour chaque valeur de `alpha` et les représenter sur une figure :

In [ ]:
plt.figure(figsize=(8, 4))

rmses = -grid.cv_results_['mean_test_score']
std_error = grid.cv_results_['std_test_score']

# afficher les RMSE avec une échelle logarithmique pour les abscisses :
plt.semilogx(grid.cv_results_['param_alpha'], rmses, 
             label="lasso", color='tab:blue')
plt.semilogx(grid.cv_results_['param_alpha'], rmses + std_error, 
             color='tab:blue', linestyle='dashed')
plt.semilogx(grid.cv_results_['param_alpha'], rmses - std_error, 
             color='tab:blue', linestyle='dashed')

# colorer l'espace entre les courbes représentant les écarts-types
plt.fill_between(alphas, (rmses + std_error), (rmses - std_error), 
                 color='tab:blue',
                 alpha=0.2, # contrôle la transparence
                )

# Mise en forme
plt.xlabel("alpha")
plt.ylabel("RMSE +/- un écart-type")
plt.xlim([alphas[0], alphas[-1]])
plt.title("Recherche sur grille (Lasso)")
plt.legend(loc='best')

__Question :__ À quoi correspond un modèle avec une faible valeur de `alpha` ? Une valeur élevée de `alpha` ?

__Réponse :__ 

#### Meilleur modèle

Les valeurs optimales de la recherche sur grille et le score correspondant sont données par les paramètres`best_params_` et `best_score_` de l'objet de la classe `GridSearchCV` :

In [ ]:
print(f"La meilleure valeur de alpha est : {grid.best_params_['alpha']:.2e}")

__Question :__ Afficher ce point sur la courbe précédente.

__Question :__ Le modèle correspondant, ré-entraîné sur l'ensemble des données passées à la fonction `fit`, est donné par le paramètre `best_estimator_`. Comparez sur une figure ses coefficients à ceux du vrai modèle.

__Réponse :__ 

## 4. Données réelles

Pour ce cas pratique nous utilisons des données cliniques. L'objectif est de **prédire le niveau d'antigène prostatique spécifique** (ou *PSA* pour *Prostate-Specific Antigen*). C'est une protéine produite exclusivement par la prostate. Un taux de concentration élevé de cette molécule dans le sang est souvent le signe chez l'homme d'un cancer de la prostate. Cet indicateur permet ainsi de suivre l'évolution d'un tel cancer.

Plus précisément, nous allons essayer de prédire le niveau de concentration du *PSA* (`lpsa`, en échelle logarithmique) à partir des mesures cliniques suivantes :
- `cavol` : Le volume de la tumeur (échelle logarithmique).
- `lweight` : Le poids de la prostate (échelle logarithmique).
- `age`: L'âge du patient.
- `lbph`: Le volume de l'hypertrophie bénigne de la prostate (*BPH* pour *Benign Prostatic Hyperplasia*) qui correspond au volume non cancéreux de l'organe (échelle logarithmique).
- `svi`: Indicateur sur le fait que le cancer s'est propagé aux vésicules séminales (deux glandes associées à la prostate).
- `lcp`: La *pénétration capsulaire* qui mesure à quel point la capsule prostatique (la membrane qui entoure la prostate), a été envahi par le cancer (échelle logarithmique).
- `gleason`: Le score de *Gleason*. Ce score est établi par un histopathologiste après observation d'une biopsie de la prostate. Pour plus d'information vous pouvez consulter ce lien : http://www.wikiwand.com/en/Gleason_grading_system. 
* `pgg45`: Le pourcentage de la tumeur qui est accrédité d'un score *Gleason* de 4 ou 5.

Ce jeu de données est un jeu de données classique, que l'on trouve par exemple [sur Kaggle](https://www.kaggle.com/tvscitechtalk/prostatecsv). Il est issu de Stamey, T.A., Kabalin, J.N., McNeal, J.E., Johnstone, I.M., Freiha, F., Redwine, E.A. and Yang, N. (1989). Prostate specific antigen in the diagnosis and treatment of adenocarcinoma of the prostate: II. radical prostatectomy treated patients, _Journal of Urology_ 141(5), 1076–1083.

Nous avons bien ici un problème de **régression**.

### 4.1 Chargement des données

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv("data/prostate.csv", index_col=0)
print(df.shape)
df.head()

Nous pouvons extraire de ce dataframe une matrice `X` de données et un vecteur `y` d'étiquettes :

In [ ]:
X = df.loc[:, ["lcavol", "lweight", "age", "lbph", "svi", "lcp", "gleason", "pgg45"]].to_numpy()
y = df.loc[:, "lpsa"].to_numpy()
print(X.shape, y.shape)

#### Jeu d'entraînement et de test

__Question :__ Séparer ce jeu de données en un jeu d'entraînement et un jeu de test (contenant 30% des observations).

In [ ]:
# split dataset for train and test
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, 
                                                                    test_size=0.3, random_state=13)

#### Transformation des variables

__Question :__ En vous inspirant de la PC4, centrez et réduisez les variables

### 4.2 Régression linéaire non régularisée

__Question :__ Calculer la RMSE d'une régression linéaire en validation croisée sur le jeu d'entraînement.

### 4.3 Régression ridge

__Question :__ En utilisant une validation croisée sur le jeu d'entraînement, effectuez une recherche sur une grille de valeurs entre $10^{-3}$ et $10^5$ du meilleur coefficient de régularisation pour une régression ridge.

__Question :__ Quelle est la valeur optimale de RMSE obtenue ? Pour quelle valeur de coefficient de régularisation ?

__Question :__ Afficher l'évolution de la RMSE en fonction de la valeur du coefficient de régularisation.

__Question :__ Afficher les coefficients de la meilleure régression ridge. Quelles variables paraissent plus pertinentes pour la prédiction ?

__Réponse :__ Les variables les plus pertinentes semblent être `lcavol`, `lweight` et `svi`.

### 4.4 Lasso

__Question :__ En utilisant une validation croisée sur le jeu d'entraînement, effectuez une recherche sur une grille de valeurs entre $10^{-4}$ et $10^2$ du meilleur coefficient de régularisation pour un lasso.

__Question :__ Quelle est la valeur optimale de RMSE obtenue ? Pour quelle valeur de coefficient de régularisation ?

__Question :__ Afficher l'évolution de la RMSE en fonction de la valeur du coefficient de régularisation.

__Question :__ Afficher sur le même graphique les coefficients de la meilleure régression ridge et ceux du meilleur lasso. Quelles variables paraissent plus pertinentes pour la prédiction ?

__Question :__ Comparer ces deux modèles.

__Réponse :__ 

__Question :__ Quel est le meilleur modèle ? 

__Réponse :__ 

__Question :__ Quelle performance peut-on espérer de ce modèle sur de nouvelles données ?

__Réponse :__ 